In [1]:
%%capture
!pip3 install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import polara
from polara import get_movielens_data
from polara.preprocessing.dataframes import leave_one_out, reindex

from dataprep import transform_indices
from evaluation import topn_recommendations, downvote_seen_items

from polara.lib.tensor import hooi

from sa_hooi import sa_hooi, form_attention_matrix, get_scaling_weights, generate_position_projector

from polara.lib.sparse import tensor_outer_at
from scipy.sparse import csr_matrix
from scipy.linalg import solve_triangular, sqrtm

%load_ext autoreload
%autoreload 2

In [6]:
def full_preproccessing(data = None):
    if data is None:
        data = get_movielens_data("ml-1m.zip", include_time=True)
    test_timepoint = data['timestamp'].quantile(
    q=0.8, interpolation='nearest'
    )
    
    if (data["rating"].nunique() > 5):
        data["rating"] = (data["rating"] * 2).astype(int)

    test_data_ = data.query('timestamp >= @test_timepoint')
    train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
    )
    
    training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')
    test_data = reindex(test_data_, data_index['items'])

    testset_, holdout_ = leave_one_out(
    test_data, target='timestamp', sample_top=True, random_state=0
    )
    testset_valid_, holdout_valid_ = leave_one_out(
        testset_, target='timestamp', sample_top=True, random_state=0
    )

    test_users_val = np.intersect1d(testset_valid_.userid.unique(), holdout_valid_.userid.unique())
    testset_valid = testset_valid_.query('userid in @test_users_val').sort_values('userid')
    holdout_valid = holdout_valid_.query('userid in @test_users_val').sort_values('userid')

    test_users = np.intersect1d(testset_.userid.unique(), holdout_.userid.unique())
    testset = testset_.query('userid in @test_users').sort_values('userid')
    holdout = holdout_.query('userid in @test_users').sort_values('userid')


    assert holdout_valid.set_index('userid')['timestamp'].ge(
        testset_valid
        .groupby('userid')
        ['timestamp'].max()
    ).all()

    data_description = dict(
        users = data_index['users'].name,
        items = data_index['items'].name,
        feedback = 'rating',
        n_users = len(data_index['users']),
        n_items = len(data_index['items']),
        n_ratings = training['rating'].nunique(),
        min_rating = training['rating'].min(),
        test_users = holdout_valid[data_index['users'].name].drop_duplicates().values, # NEW
        n_test_users = holdout_valid[data_index['users'].name].nunique() # NEW
    )

    return training, testset_valid, holdout_valid, testset, holdout, data_description, data_index

In [8]:
training, testset_valid, holdout_valid, testset, holdout, data_description, data_index = full_preproccessing()

Filtered 177 invalid observations.


In [10]:
from IPython.utils import io
from scipy.special import softmax

def tf_model_build(config, data, data_description, testset, holdout, attention_matrix=np.array([])):
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    idx = data[[userid, itemid, feedback]].values
    idx[:, -1] = idx[:, -1] - data_description['min_rating'] # works only for integer ratings!
    val = np.ones(idx.shape[0], dtype='f8')
    
    n_users = data_description["n_users"]
    n_items = data_description["n_items"]
    n_ratings = data_description["n_ratings"]
    shape = (n_users, n_items, n_ratings)
    core_shape = config['mlrank']
    num_iters = config["num_iters"]
    
    if (attention_matrix.shape[0] == 0):
        attention_matrix = form_attention_matrix(
            data_description['n_ratings'],
            **config['params'],
            format = 'csr'
        )
        
    attention_matrix = np.array(attention_matrix)

    item_popularity = (
        data[itemid]
        .value_counts(sort=False)
        .reindex(range(n_items))
        .fillna(1)
        .values
    )
    scaling_weights = get_scaling_weights(item_popularity, scaling=config["scaling"])

    with io.capture_output() as captured:
        u0, u1, u2 = sa_hooi(
            idx, val, shape, config["mlrank"],
            attention_matrix = attention_matrix,
            scaling_weights = scaling_weights,
            testset = testset,
            holdout = holdout,
            data_description = data_description,
            max_iters = config["num_iters"],
            parallel_ttm = True,
            randomized = config["randomized"],
            growth_tol = config["growth_tol"],
            seed = config["seed"],
            iter_callback = None,
        )
    
    return u0, u1, u2, attention_matrix    
    
config = {
    "scaling": 1,
    "mlrank": (30, 30, data_description['n_ratings']),
    "n_ratings": data_description['n_ratings'],
    "num_iters": 5,
    "params": None,
    "randomized": True,
    "growth_tol": 1e-4,
    "seed": 42
}

def tf_scoring(params, data, data_description, context=["3+4+5"]):
    user_factors, item_factors, feedback_factors, attention_matrix = params
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    data = data.sort_values(userid)
    useridx = data[userid]
    itemidx = data[itemid].values
    ratings = data[feedback].values
    ratings = ratings - data_description['min_rating'] # NEW
    
    n_users = useridx.nunique()
    n_items = data_description['n_items']
    n_ratings = data_description['n_ratings']
    
    inv_attention = solve_triangular(attention_matrix, np.eye(n_ratings), lower=True)
    
    tensor_outer = tensor_outer_at('cpu')
    #matrix_softmax = softmax(inv_attention.T @ feedback_factors)
    matrix_softmax = inv_attention.T @ feedback_factors
    #
    if (n_ratings == 10):
        coef = 2
    else:
        coef = 1
        
    if (context == "5"): # make softmax 
        inv_aT_feedback = matrix_softmax[(-1 * coef) , :]
    elif (context == "4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[(-2 * coef):, :], axis=0)
    elif (context == "3+4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[(-3 * coef):, :], axis=0)
    #elif (context == "2+3+4+5"):
    #    inv_aT_feedback = np.sum(matrix_softmax[-4:, :], axis=0)
    elif (context == "3+4+5-2-1"):
        inv_aT_feedback = np.sum(matrix_softmax[(-3 * coef):, :], axis=0) - np.sum(matrix_softmax[:(2 * coef), :], axis=0)
        
    scores = tensor_outer(
        1.0,
        item_factors,
        attention_matrix @ feedback_factors,
        itemidx,
        ratings
    )
    scores = np.add.reduceat(scores, np.r_[0, np.where(np.diff(useridx))[0]+1]) # sort by users
    scores = np.tensordot(
        scores,
        inv_aT_feedback,
        axes=(2, 0)
    ).dot(item_factors.T)

    return scores


def model_evaluate(recommended_items, holdout, holdout_description, alpha=3, topn=10, dcg=False):
    itemid = holdout_description['items']
    rateid = holdout_description['feedback']
    alpha = 3 if holdout_description["n_ratings"] == 5 else 6
    n_test_users = recommended_items.shape[0]
    holdout_items = holdout[itemid].values
    assert recommended_items.shape[0] == len(holdout_items)
    
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    pos_mask = (holdout[rateid] >= alpha).values
    neg_mask = (holdout[rateid] < alpha).values
    
    # HR calculation
    #hr = np.sum(hits_mask.any(axis=1)) / n_test_users
    hr_pos = np.sum(hits_mask[pos_mask].any(axis=1)) / n_test_users
    hr_neg = np.sum(hits_mask[neg_mask].any(axis=1)) / n_test_users
    hr = hr_pos + hr_neg
    
    # MRR calculation
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / n_test_users
    pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
    mrr_pos = np.sum(1 / pos_hit_rank) / n_test_users
    neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
    mrr_neg = np.sum(1 / neg_hit_rank) / n_test_users
    
    # Matthews correlation
    TP = np.sum(hits_mask[pos_mask]) # + 
    FP = np.sum(hits_mask[neg_mask]) # +
    cond = (hits_mask.sum(axis = 1) == 0)
    FN = np.sum(cond[pos_mask])
    TN = np.sum(cond[neg_mask])
    N = TP+FP+TN+FN
    S = (TP+FN)/N
    P = (TP+FP)/N
    C = (TP/N - S*P) / np.sqrt(P*S*(1-P)*(1-S))
    
    # DCG calculation
    if dcg:
        pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
        neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
        ndcg = np.mean(1 / np.log2(pos_hit_rank+1))
        ndcl = np.mean(1 / np.log2(neg_hit_rank+1))
    
    # coverage calculation
    n_items = holdout_description['n_items']
    cov = np.unique(recommended_items).size / n_items
    if dcg:
        return hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C, ndcg, ndcl
    else:
        return hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C

def make_prediction(tf_scores, holdout, data_description, mode, context="", print_mode=True):
    if (mode and print_mode):
        print(f"for context {context} evaluation ({mode}): \n")
    for n in [5, 10, 20]:
        tf_recs = topn_recommendations(tf_scores, n)
        hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C = model_evaluate(tf_recs, holdout, data_description, topn=n)
        if (print_mode):
            print(f"HR@{n} = {hr:.4f}, MRR@{n} = {mrr:.4f}, Coverage@{n} = {cov:.4f}")
            print(f"HR_pos@{n} = {hr_pos:.4f}, HR_neg@{n} = {hr_neg:.4f}")
            print(f"MRR_pos@{n} = {mrr_pos:.4f}, MRR_neg@{n} = {mrr_neg:.4f}")
            print(f"Matthews@{n} = {C:.4f}")
            print("-------------------------------------")
        if (n == 10):
            mrr10 = mrr
            hr10 = hr
            c10 = C
    return mrr10, hr10, c10

def valid_mlrank(mlrank):
    '''
    Only allow ranks that are suitable for truncated SVD computations
    on unfolded compressed tensor (the result of ttm product in HOOI).
    '''
    #s, r1, r2, r3 = mlrank
    s, r1, r3 = mlrank
    r2 = r1
    #print(s, r1, r2, r3)
    return r1*r2 > r3 and r1*r3 > r2 and r2*r3 > r1

In [11]:
from tqdm import tqdm 
from polara.evaluation.pipelines import random_grid

from tqdm import tqdm 
from polara.evaluation.pipelines import random_grid

def full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix):

    config["mlrank"] = (64, 64, data_description["n_ratings"])
    print("Starting pipeline...")
    print("Training with different context in progress...")
    print("------------------------------------------------------")

    for context in ["5", "4+5", "3+4+5", "3+4+5-2-1"]:
        tf_params = tf_model_build(config, training, data_description, testset_valid, holdout_valid, attention_matrix=attention_matrix)
        seen_data = testset_valid
        tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
        downvote_seen_items(tf_scores, seen_data, data_description)
        cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout_valid, data_description, "Validation", context)
        print("------------------------------------------------------")

    print(f"Tuning model for all contexts...\n")

    rank_grid = []
    for i in range(5, 7):
        rank_grid.append(2 * 2 ** i)
        rank_grid.append(3 * 2 ** i)
    
    rank_grid = np.array(rank_grid)
    tf_hyper = {
    'scaling': [0.7, 1.9], #np.linspace(0, 2, 21),
    'r1': rank_grid, #np.arange(100, 220, 25),
    #'r2': np.arange(50, 801, 25),
    'r3': range(1, 6, 1)#range(2, 11, 2), # change
    }

    grid, param_names = random_grid(tf_hyper, n=0)
    tf_grid = [tuple(mlrank) for mlrank in grid if valid_mlrank(mlrank)]

    hr_tf = {}
    hr_pos_tf = {}
    hr_neg_tf = {}
    mrr_tf = {}
    mrr_pos_tf = {}
    mrr_neg_tf = {}
    cov_tf = {}
    C_tf = {}
    
    seen_data = testset_valid
    
    for mlrank in tqdm(tf_grid):
        with io.capture_output() as captured:
            r1, r3 = mlrank[1:]
            cur_mlrank = tuple((r1, r1, r3))
            config['mlrank'] = cur_mlrank
            config['scaling'] = mlrank[0]
            tf_params = tf_model_build(config, training, data_description, testset_valid, holdout_valid, attention_matrix=attention_matrix)
            for context in ["5", "4+5", "3+4+5", "3+4+5-2-1"]:
                tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
                downvote_seen_items(tf_scores, seen_data, data_description)
                tf_recs = topn_recommendations(tf_scores, topn=10)
                
                hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C = model_evaluate(tf_recs, holdout_valid, data_description, topn=10)
                hr_tf[(context, cur_mlrank, mlrank[0])] = hr
                hr_pos_tf[(context, cur_mlrank, mlrank[0])] = hr_pos
                hr_neg_tf[(context, cur_mlrank, mlrank[0])] = hr_neg
                mrr_tf[(context, cur_mlrank, mlrank[0])] = mrr
                mrr_pos_tf[(context, cur_mlrank, mlrank[0])] = mrr_pos
                mrr_neg_tf[(context, cur_mlrank, mlrank[0])] = mrr_neg
                cov_tf[(context, cur_mlrank, mlrank[0])] = cov
                C_tf[(context, cur_mlrank, mlrank[0])] = C

    print(f'Best HR={pd.Series(hr_tf).max():.4f} achieved with context {pd.Series(hr_tf).idxmax()[0]} and mlrank = {pd.Series(hr_tf).idxmax()[1]} and scale factor = {pd.Series(hr_tf).idxmax()[2]}')
    print(f'Best HR_pos={pd.Series(hr_pos_tf).max():.4f} achieved with context {pd.Series(hr_pos_tf).idxmax()[0]} and mlrank = {pd.Series(hr_pos_tf).idxmax()[1]} and scale factor = {pd.Series(hr_pos_tf).idxmax()[2]}')
    print(f'Best HR_neg={pd.Series(hr_neg_tf).min():.4f} achieved with context {pd.Series(hr_neg_tf).idxmin()[0]} and mlrank = {pd.Series(hr_neg_tf).idxmin()[1]} and scale factor = {pd.Series(hr_neg_tf).idxmin()[2]}')
    
    print(f'Best MRR={pd.Series(mrr_tf).max():.4f} achieved with context {pd.Series(mrr_tf).idxmax()[0]} and mlrank = {pd.Series(mrr_tf).idxmax()[1]} and scale factor = {pd.Series(mrr_tf).idxmax()[2]}')
    print(f'Best MRR_pos={pd.Series(mrr_pos_tf).max():.4f} achieved with context {pd.Series(mrr_pos_tf).idxmax()[0]} and mlrank = {pd.Series(mrr_pos_tf).idxmax()[1]} and scale factor = {pd.Series(mrr_pos_tf).idxmax()[2]}')
    print(f'Best MRR_neg={pd.Series(mrr_neg_tf).min():.4f} achieved with context {pd.Series(mrr_neg_tf).idxmin()[0]} and mlrank = {pd.Series(mrr_neg_tf).idxmin()[1]} and scale factor = {pd.Series(mrr_neg_tf).idxmin()[2]}')
    
    print(f'Best Matthews={pd.Series(C_tf).max():.4f} achieved with context {pd.Series(C_tf).idxmax()[0]} and mlrank = {pd.Series(C_tf).idxmax()[1]} and scale factor = {pd.Series(C_tf).idxmax()[2]}')
                          
    print(f'COV={pd.Series(cov_tf)[pd.Series(C_tf).idxmax()]:.4f} (based on best Matthews value)')
    print("---------------------------------------------------------")
    print("Evaluation of the best model on test holdout in progress...\n")
    
    print("Best by MRR@10:\n")
    config["mlrank"] = pd.Series(mrr_pos_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, testset, holdout, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(mrr_pos_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(mrr_pos_tf).idxmax()[0])
    
    print("---------------------------------------------------------")
    
    print("Best by HR@10:\n")
    config["mlrank"] = pd.Series(hr_pos_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, testset, holdout, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(hr_pos_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(hr_pos_tf).idxmax()[0])
    
    print("---------------------------------------------------------")
    
    print("Best by Matthews@10:\n")
    config["mlrank"] = pd.Series(C_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, testset, holdout, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(C_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(C_tf).idxmax()[0])
    print("Pipeline ended.")

# Custom attention

In [16]:
similarity_matrix = np.array([[1.0, 0.85, 0.2, 0.01, 0.01], [0.85, 1.0, 0.7, 0.2, 0.01], [0.2, 0.7, 1.0, 0.65, 0.5], [0.01, 0.2, 0.65, 1.0, 0.85], [0.01, 0.01, 0.5, 0.85, 1.0]])
similarity_matrix

array([[1.  , 0.85, 0.2 , 0.01, 0.01],
       [0.85, 1.  , 0.7 , 0.2 , 0.01],
       [0.2 , 0.7 , 1.  , 0.65, 0.5 ],
       [0.01, 0.2 , 0.65, 1.  , 0.85],
       [0.01, 0.01, 0.5 , 0.85, 1.  ]])

In [17]:
config["params"] = {}
attention_matrix = sqrtm(similarity_matrix).real
#without sigmoid
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------
for context 5 evaluation (Validation): 

HR@5 = 0.0422, MRR@5 = 0.0197, Coverage@5 = 0.1199
HR_pos@5 = 0.0411, HR_neg@5 = 0.0012
MRR_pos@5 = 0.0194, MRR_neg@5 = 0.0003
Matthews@5 = 0.0787
-------------------------------------
HR@10 = 0.0610, MRR@10 = 0.0222, Coverage@10 = 0.1670
HR_pos@10 = 0.0592, HR_neg@10 = 0.0018
MRR_pos@10 = 0.0218, MRR_neg@10 = 0.0003
Matthews@10 = 0.0947
-------------------------------------
HR@20 = 0.1138, MRR@20 = 0.0258, Coverage@20 = 0.2278
HR_pos@20 = 0.1044, HR_neg@20 = 0.0094
MRR_pos@20 = 0.0249, MRR_neg@20 = 0.0008
Matthews@20 = 0.0818
-------------------------------------
------------------------------------------------------
for context 4+5 evaluation (Validation): 

HR@5 = 0.0370, MRR@5 = 0.0197, Coverage@5 = 0.1280
HR_pos@5 = 0.0352, HR_neg@5 = 0.0018
MRR_pos@5 = 0.0183, MRR_neg@5 = 0.0015
Matthews@5 = 0.0630
--------------------

100%|██████████| 420/420 [2:28:13<00:00, 21.17s/it]  


Best HR=0.0997 achieved with context 2+3+4+5 and mlrank = (200, 200, 4) and scale factor = 0.7000000000000001
Best HR_pos=0.0856 achieved with context 2+3+4+5 and mlrank = (200, 200, 4) and scale factor = 0.8
Best HR_neg=0.0023 achieved with context 5 and mlrank = (175, 175, 2) and scale factor = 2.0
Best MRR=0.0357 achieved with context 2+3+4+5 and mlrank = (125, 125, 5) and scale factor = 0.0
Best MRR_pos=0.0329 achieved with context 3+4+5 and mlrank = (175, 175, 3) and scale factor = 0.0
Best MRR_neg=0.0004 achieved with context 5 and mlrank = (100, 100, 4) and scale factor = 0.8
Best Matthews=0.1120 achieved with context 5 and mlrank = (100, 100, 5) and scale factor = 0.9
COV=0.2312 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 3+4+5 evaluation (Test): 

HR@5 = 0.0362, MRR@5 = 0.0175, Coverage@5 = 0.2161
HR_pos@5 = 0.0305, HR_neg@5 = 0.0058
MRR_pos@5 

In [ ]:
# test sigmoid without normalization (scale factor)

## Attention matrices

In [16]:
def sigmoid_func(x):
    return 1.0 / (1 + np.exp(-x))

def arctan(x):
    return 0.5 * np.arctan(x) + 0.5

def sq3(x):
    return 0.5 * np.cbrt(x) + 0.5

def get_similarity_matrix(mode, n_ratings = 10):
    matrix = np.zeros((n_ratings, n_ratings))
    if (mode == "sigmoid"):
        x_space = np.linspace(-6, 6, n_ratings)
        for i in range(n_ratings):
            for j in range(i, n_ratings, 1):
                matrix[i, j] = 1.0 - np.abs(sigmoid_func(x_space[i]) - sigmoid_func(x_space[j]))
                matrix[j, i] = matrix[i, j]
                
    elif (mode == "linear"):
        x_space = np.linspace(0, 1, n_ratings)
        for i in range(n_ratings):
            for j in range(i, n_ratings, 1):
                matrix[i, j] = 1.0 - np.abs(x_space[i] - x_space[j])
                matrix[j, i] = matrix[i, j]
                
    elif (mode == "arctan"):
        x_space = np.linspace(-np.pi / 2.0, np.pi / 2.0, n_ratings)
        for i in range(n_ratings):
            for j in range(i, n_ratings, 1):
                matrix[i, j] = 1.0 - np.abs(arctan(x_space[i]) - arctan(x_space[j]))
                matrix[j, i] = matrix[i, j]
                
    elif (mode == "sq3"):
        x_space = np.linspace(-1, 1, n_ratings)
        for i in range(n_ratings):
            for j in range(i, n_ratings, 1):
                matrix[i, j] = 1.0 - np.abs(sq3(x_space[i]) - sq3(x_space[j]))
                matrix[j, i] = matrix[i, j]
                
    return matrix

In [17]:
matrices = []
config["params"] = {}
modes = ["sigmoid", "linear", "arctan", "sq3"]

for mode in modes:
    print(f"FOR matrix {mode}")
    similarity_matrix = get_similarity_matrix(mode, data_description["n_ratings"])
    attention_matrix = sqrtm(similarity_matrix).real
    full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=attention_matrix)

FOR matrix sigmoid
Starting pipeline...
Training with different context in progress...
------------------------------------------------------
for context 5 evaluation (Validation): 

HR@5 = 0.0399, MRR@5 = 0.0213, Coverage@5 = 0.1584
HR_pos@5 = 0.0393, HR_neg@5 = 0.0006
MRR_pos@5 = 0.0212, MRR_neg@5 = 0.0001
Matthews@5 = 0.0835
-------------------------------------
HR@10 = 0.0774, MRR@10 = 0.0264, Coverage@10 = 0.2122
HR_pos@10 = 0.0733, HR_neg@10 = 0.0041
MRR_pos@10 = 0.0257, MRR_neg@10 = 0.0006
Matthews@10 = 0.0889
-------------------------------------
HR@20 = 0.1296, MRR@20 = 0.0298, Coverage@20 = 0.2842
HR_pos@20 = 0.1208, HR_neg@20 = 0.0088
MRR_pos@20 = 0.0289, MRR_neg@20 = 0.0009
Matthews@20 = 0.1031
-------------------------------------
------------------------------------------------------
for context 4+5 evaluation (Validation): 

HR@5 = 0.0446, MRR@5 = 0.0244, Coverage@5 = 0.1590
HR_pos@5 = 0.0422, HR_neg@5 = 0.0023
MRR_pos@5 = 0.0234, MRR_neg@5 = 0.0010
Matthews@5 = 0.0665
-

100%|██████████| 32/32 [08:00<00:00, 15.01s/it]


Best HR=0.0880 achieved with context 3+4+5-2-1 and mlrank = (192, 192, 3) and scale factor = 0.7
Best HR_pos=0.0792 achieved with context 3+4+5 and mlrank = (192, 192, 4) and scale factor = 0.7
Best HR_neg=0.0018 achieved with context 5 and mlrank = (64, 64, 5) and scale factor = 1.9
Best MRR=0.0337 achieved with context 3+4+5 and mlrank = (128, 128, 4) and scale factor = 0.7
Best MRR_pos=0.0308 achieved with context 4+5 and mlrank = (192, 192, 5) and scale factor = 0.7
Best MRR_neg=0.0004 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 0.7
Best Matthews=0.1070 achieved with context 5 and mlrank = (64, 64, 5) and scale factor = 1.9
COV=0.1905 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 4+5 evaluation (Test): 

HR@5 = 0.0426, MRR@5 = 0.0223, Coverage@5 = 0.2153
HR_pos@5 = 0.0380, HR_neg@5 = 0.0046
MRR_pos@5 = 0.0194, MRR_neg@5 = 0.002

100%|██████████| 32/32 [08:34<00:00, 16.07s/it]


Best HR=0.0927 achieved with context 3+4+5 and mlrank = (192, 192, 2) and scale factor = 0.7
Best HR_pos=0.0827 achieved with context 5 and mlrank = (192, 192, 5) and scale factor = 0.7
Best HR_neg=0.0018 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 1.9
Best MRR=0.0337 achieved with context 3+4+5 and mlrank = (128, 128, 5) and scale factor = 0.7
Best MRR_pos=0.0306 achieved with context 4+5 and mlrank = (192, 192, 3) and scale factor = 0.7
Best MRR_neg=0.0006 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 0.7
Best Matthews=0.1106 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 1.9
COV=0.1866 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 4+5 evaluation (Test): 

HR@5 = 0.0408, MRR@5 = 0.0205, Coverage@5 = 0.2183
HR_pos@5 = 0.0368, HR_neg@5 = 0.0040
MRR_pos@5 = 0.0189, MRR_neg@5 = 0.0017
Matthe

100%|██████████| 32/32 [08:14<00:00, 15.44s/it]


Best HR=0.0915 achieved with context 3+4+5 and mlrank = (192, 192, 4) and scale factor = 0.7
Best HR_pos=0.0809 achieved with context 4+5 and mlrank = (128, 128, 4) and scale factor = 0.7
Best HR_neg=0.0018 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 1.9
Best MRR=0.0335 achieved with context 4+5 and mlrank = (192, 192, 5) and scale factor = 0.7
Best MRR_pos=0.0315 achieved with context 3+4+5-2-1 and mlrank = (192, 192, 4) and scale factor = 0.7
Best MRR_neg=0.0005 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 0.7
Best Matthews=0.1047 achieved with context 5 and mlrank = (64, 64, 3) and scale factor = 1.9
COV=0.1933 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 3+4+5-2-1 evaluation (Test): 

HR@5 = 0.0431, MRR@5 = 0.0208, Coverage@5 = 0.2276
HR_pos@5 = 0.0391, HR_neg@5 = 0.0040
MRR_pos@5 = 0.0187, MRR_neg@5 = 0

100%|██████████| 32/32 [08:46<00:00, 16.45s/it]


Best HR=0.0903 achieved with context 3+4+5 and mlrank = (192, 192, 4) and scale factor = 0.7
Best HR_pos=0.0815 achieved with context 3+4+5 and mlrank = (192, 192, 4) and scale factor = 0.7
Best HR_neg=0.0023 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 0.7
Best MRR=0.0345 achieved with context 3+4+5 and mlrank = (192, 192, 3) and scale factor = 0.7
Best MRR_pos=0.0316 achieved with context 3+4+5 and mlrank = (128, 128, 5) and scale factor = 0.7
Best MRR_neg=0.0003 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 0.7
Best Matthews=0.1072 achieved with context 5 and mlrank = (64, 64, 2) and scale factor = 0.7
COV=0.2105 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 3+4+5 evaluation (Test): 

HR@5 = 0.0414, MRR@5 = 0.0208, Coverage@5 = 0.2083
HR_pos@5 = 0.0362, HR_neg@5 = 0.0052
MRR_pos@5 = 0.0188, MRR_neg@5 = 0.002